## All my data

The purpose of this analysis is to bring together all of the metrics that I've gathered on food and exercise over the past year or so and do something clever with it...

### First task: get your Strava data

I got the setup from [here](https://towardsdatascience.com/using-the-strava-api-and-pandas-to-explore-your-activity-data-d94901d9bfde)


Actually, this [youtube video](f5ff8b447b7ae7882c6ffb728d16b97480e0d2c6) got me where I needed to be. 

There was a trick that I didn't see written anywhere, and that is the granting of read all access one time... but, now that that's done, I should be good for life...

In [2]:
import requests
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import sys
sys.path.insert(1, '/content/drive/MyDrive/py_deps')
import creds

In [5]:
auth_url = "https://www.strava.com/oauth/token"
activites_url = "https://www.strava.com/api/v3/athlete/activities"

#replace variables with values for your account
payload = {
    'client_id': creds.STRAVA_CLIENT_ID,
    'client_secret': creds.STRAVA_CLIENT_SECRET,
    'refresh_token': creds.STRAVA_REFRESH_TOKEN,
    'grant_type': "refresh_token",
    'f': 'json'
}


In [6]:
print("Requesting Token...\n")
res = requests.post(auth_url, data=payload, verify=False)
access_token = res.json()['access_token']
print("Access Token = {}\n".format(access_token))

Requesting Token...

Access Token = 0ec933a33de0427b056670f597cfdcd18462252b



In [7]:
header = {'Authorization': 'Bearer ' + access_token}
param = {'per_page': 200, 'page': 1}
my_dataset = requests.get(activites_url, headers=header, params=param).json()

In [8]:
import pandas as pd
from pandas.io.json import json_normalize
activities = json_normalize(my_dataset)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
header = {'Authorization': 'Bearer ' + access_token}
param = {'per_page': 200, 'page': 2}
my_dataset = requests.get(activites_url, headers=header, params=param).json()

In [10]:
activities2 = json_normalize(my_dataset)
activities2.start_date_local.min()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


'2018-06-18T11:54:37Z'

In [11]:
activities2.describe()


,resource_state,distance,moving_time,elapsed_time,total_elevation_gain,workout_type,id,utc_offset,achievement_count,kudos_count,...,map.resource_state,elev_high,elev_low,average_cadence,average_heartrate,max_heartrate,suffer_score,average_watts,kilojoules,average_temp
count,194.0,194.000000,194.000000,1.940000e+02,194.000000,66.000000,1.940000e+02,194.000000,194.000000,194.000000,...,194.0,156.000000,156.000000,123.000000,98.000000,98.000000,97.000000,6.000000,6.000000,33.000000
mean,2.0,5349.719588,2101.463918,2.975260e+04,29.581959,0.348485,4.419315e+09,-15624.742268,1.649485,0.298969,...,2.0,39.081410,4.457051,72.847967,124.723469,143.622449,31.412371,27.033333,88.400000,24.606061
std,0.0,4172.981999,1249.342986,3.109539e+05,45.255335,1.758430,1.182989e+09,2129.731839,3.718157,0.532176,...,0.0,100.410781,90.936083,15.046455,29.512771,28.487478,48.422781,11.476701,117.586921,3.605026
min,2.0,0.000000,0.000000,1.300000e+01,0.000000,0.000000,1.647040e+09,-25200.000000,0.000000,0.000000,...,2.0,-350.200000,-356.000000,19.000000,87.600000,95.000000,0.000000,16.300000,17.400000,17.000000
25%,2.0,1602.225000,1291.750000,1.491250e+03,0.000000,0.000000,3.865224e+09,-18000.000000,0.000000,0.000000,...,2.0,17.425000,-4.100000,58.650000,101.325000,120.000000,2.000000,19.800000,37.225000,23.000000
50%,2.0,5647.150000,2137.500000,2.406000e+03,10.000000,0.000000,4.591739e+09,-14400.000000,0.000000,0.000000,...,2.0,37.300000,5.450000,76.800000,113.050000,136.000000,4.000000,24.850000,44.950000,25.000000
75%,2.0,8046.700000,2668.250000,3.096250e+03,40.250000,0.000000,5.512534e+09,-14400.000000,1.000000,1.000000,...,2.0,58.350000,17.400000,87.400000,161.650000,174.000000,62.000000,28.625000,57.625000,27.000000
max,2.0,30973.500000,7659.000000,4.189958e+06,292.000000,10.000000,5.821046e+09,-14400.000000,26.000000,2.000000,...,2.0,414.000000,289.200000,92.100000,174.600000,194.000000,159.000000,48.200000,326.600000,31.000000


In [12]:
allActivities = pd.concat([activities, activities2], ignore_index=True, axis=0)

In [13]:
allActivities.describe()

,resource_state,distance,moving_time,elapsed_time,total_elevation_gain,workout_type,id,utc_offset,achievement_count,kudos_count,...,upload_id,pr_count,total_photo_count,athlete.id,athlete.resource_state,map.resource_state,average_heartrate,max_heartrate,suffer_score,average_temp
count,394.0,394.000000,394.000000,3.940000e+02,394.000000,207.000000,3.940000e+02,394.000000,394.000000,394.000000,...,2.970000e+02,394.000000,394.000000,394.0,394.0,394.0,113.000000,113.000000,112.000000,33.000000
mean,2.0,8707.956091,3221.456853,1.705754e+04,44.107107,0.270531,5.573926e+09,-15861.928934,2.129442,0.474619,...,5.724409e+09,0.857868,0.071066,17913312.0,1.0,2.0,129.100885,148.212389,47.866071,24.606061
std,0.0,7511.640129,2944.568007,2.182878e+05,100.821916,1.562377,1.451581e+09,2625.462002,4.405756,0.856417,...,1.647449e+09,2.520168,0.422070,0.0,0.0,0.0,30.071642,29.307316,74.325980,3.605026
min,2.0,0.000000,0.000000,1.300000e+01,0.000000,0.000000,1.647040e+09,-36000.000000,0.000000,0.000000,...,2.096164e+09,0.000000,0.000000,17913312.0,1.0,2.0,87.600000,95.000000,0.000000,17.000000
25%,2.0,3784.850000,1803.250000,2.075750e+03,0.000000,0.000000,4.596848e+09,-18000.000000,0.000000,0.000000,...,4.534439e+09,0.000000,0.000000,17913312.0,1.0,2.0,102.500000,123.000000,2.000000,23.000000
50%,2.0,8046.700000,2688.500000,2.990000e+03,16.150000,0.000000,5.845387e+09,-14400.000000,0.000000,0.000000,...,5.882212e+09,0.000000,0.000000,17913312.0,1.0,2.0,116.700000,142.000000,6.500000,25.000000
75%,2.0,11265.400000,3756.250000,4.197750e+03,61.750000,0.000000,6.726428e+09,-14400.000000,2.000000,1.000000,...,7.220805e+09,0.000000,0.000000,17913312.0,1.0,2.0,163.800000,178.000000,89.250000,27.000000
max,2.0,81715.000000,39346.000000,4.189958e+06,1286.000000,10.000000,7.566527e+09,-14400.000000,26.000000,6.000000,...,8.069749e+09,26.000000,3.000000,17913312.0,1.0,2.0,179.200000,199.000000,420.000000,31.000000


So, what features do I want?

- Date features



In [14]:
# convert distance from meters to miles
allActivities['distance_mi'] = allActivities['distance']/1609

In [15]:
allActivities.distance_mi.describe()

count    394.000000
mean       5.412030
std        4.668515
min        0.000000
25%        2.352300
50%        5.001057
75%        7.001492
max       50.786203
Name: distance_mi, dtype: float64

In [16]:

allActivities.sport_type.unique()

array(['Run', 'Hike', 'Workout', 'Yoga', 'Ride', 'Walk', 'RockClimbing',
       'Swim'], dtype=object)

In [17]:
runs = allActivities.query('sport_type == "Run"')


In [18]:
print(f'{runs.start_date_local.min()} to {runs.start_date_local.max()}')

2018-06-18T11:54:37Z to 2022-08-01T15:51:30Z


---
OK, what's next?  Visualize or bring in food diary?

In [20]:
runs.columns

Index(['resource_state', 'name', 'distance', 'moving_time', 'elapsed_time',
       'total_elevation_gain', 'type', 'sport_type', 'workout_type', 'id',
       'start_date', 'start_date_local', 'timezone', 'utc_offset',
       'location_city', 'location_state', 'location_country',
       'achievement_count', 'kudos_count', 'comment_count', 'athlete_count',
       'photo_count', 'trainer', 'commute', 'manual', 'private', 'visibility',
       'flagged', 'gear_id', 'start_latlng', 'end_latlng', 'average_speed',
       'max_speed', 'average_cadence', 'average_watts', 'max_watts',
       'weighted_average_watts', 'kilojoules', 'device_watts', 'has_heartrate',
       'heartrate_opt_out', 'display_hide_heartrate_option', 'elev_high',
       'elev_low', 'upload_id', 'upload_id_str', 'external_id',
       'from_accepted_tag', 'pr_count', 'total_photo_count', 'has_kudoed',
       'athlete.id', 'athlete.resource_state', 'map.id',
       'map.summary_polyline', 'map.resource_state', 'average_heartra

In [21]:
pd.set_option('display.max_rows', None)
runs.dtypes

resource_state                     int64
name                              object
distance                         float64
moving_time                        int64
elapsed_time                       int64
total_elevation_gain             float64
type                              object
sport_type                        object
workout_type                     float64
id                                 int64
start_date                        object
start_date_local                  object
timezone                          object
utc_offset                       float64
location_city                     object
location_state                    object
location_country                  object
achievement_count                  int64
kudos_count                        int64
comment_count                      int64
athlete_count                      int64
photo_count                        int64
trainer                             bool
commute                             bool
manual          

In [22]:
runs.start_date.head()

0    2022-08-01T19:51:30Z
3    2022-07-15T15:58:33Z
4    2022-07-13T20:27:44Z
5    2022-07-12T01:13:04Z
6    2022-07-10T20:08:00Z
Name: start_date, dtype: object

In [24]:
runs.start_date = pd.to_datetime(runs.start_date, utc=True)
runs.start_date_local = pd.to_datetime(runs.start_date_local, utc=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [33]:
runs['start_month'] = runs.start_date_local.dt.month

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [34]:
runs.head()

,resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,id,...,athlete.resource_state,map.id,map.summary_polyline,map.resource_state,average_heartrate,max_heartrate,suffer_score,average_temp,distance_mi,start_month
0,2,Afternoon Run,11571.5,4484,4896,85.0,Run,Run,NaN,7566527077,...,1,a7566527077,{iclFdqluMwBsAiAg@w@e@c@[q@w@]YcAe@m@Gg@OyBaBq...,2,NaN,NaN,NaN,NaN,7.191734,8
3,2,Lunch Run,28976.0,10774,12782,191.0,Run,Run,NaN,7474497158,...,1,a7474497158,gxalFr|nuM{@fIq@bEL`ACXsB~JmDvMsCtGa@zAeAjC{A`...,2,NaN,NaN,NaN,NaN,18.008701,7
4,2,Speed work with warm/cool,12065.0,4313,4774,44.0,Run,Run,0.0,7464617290,...,1,a7464617290,i~alF~qjuMKKEU@IR_@DS@SEs@?ILa@L}@Xg@Z_@Tu@p@o...,2,NaN,NaN,NaN,NaN,7.498446,7
5,2,Post strength fatigue dreadmill,8060.6,2588,2588,0.0,Run,Run,0.0,7454481074,...,1,a7454481074,None,2,NaN,NaN,NaN,NaN,5.009695,7
6,2,Afternoon Run,25754.0,9132,10904,190.0,Run,Run,0.0,7448725696,...,1,a7448725696,}vykFvjuuMYkBIoB{@eGEwCuAmOFk@o@BMKk@gH?mDPoCp...,2,NaN,NaN,NaN,NaN,16.006215,7
